# Quiz \#2: Text Generation

## Simple text generator with Markov chains

In this short quiz, you'll build a simple Markov chain text generator with regular expressions. A Markov chain text generator works by randomly sitching words together. Regular expressions are a powerful way to find matches in a text, so they'll help you find good words to randomly stitch together.

* Garkov, a Markov chain Garfield generator: http://joshmillard.com/garkov/
* Python regular expressions: https://docs.python.org/3/library/re.html
    * If you Google for images of "regex cheatsheet" you'll find a lot of helpful stuff.

This quiz will use the Emma corpus from the `nltk` Python library. It's a free novel that's easy to get.

* Web version of Emma by Jane Austen: https://www.gutenberg.org/files/158/158-h/158-h.htm
* Wikipedia: https://en.wikipedia.org/wiki/Emma_(novel)

## Example output

Can you tell the difference between real and fake? The generated text has a weakness unfortunately: it has a short memory.

### Real

```
The affection of the whole family, the warm attachment of Miss Campbell in particular, was the more honourable to each party from the circumstance of Jane's decided superiority both in beauty and acquirements. That nature had given it in feature could not be unseen by the young woman, nor could her higher powers of mind be unfelt by the parents. They continued together with unabated regard however, till the marriage of Miss Campbell, who by that chance, that luck which so often defies anticipation in matrimonial affairs, giving attraction to what is moderate rather than to what is superior, engaged the affections of Mr. Dixon, a young man, rich and agreeable, almost as soon as they were acquainted; and was eligibly and happily settled, while Jane Fairfax had yet her bread to earn.
```

### Fake

```
"What was to be at all; and his sister's side, Emma proposed it to Harriet, as they were acquainted; and was repeating, "No husbands and wives in the case, my dear aunt; there is no subject of hours of the twenty-four to his comfort, while Mrs. Weston pays her visit, and describe the many to be cared for.-- I hope you are going farther on business three days ago, and to have been drawn on beyond her own playing and singing.  She did most heartily grieve over the advantages of sitting still when he comes again, if his affection continue, I must speak to you again."
```

As you can see, the generated text has bad punctuation and non-matching quotation marks.

## How to use this quiz

### On your computer

* Instructions: https://jupyter.org/install

If you have basic Python with pip, use `pip install jupyterlab` to install Jupyter lab. You can then download with `.ipynb` file to your computer, and then start `jupyter lab` to open it.

If you have Anaconda, you can install Jupyter with `conda install -c conda-forge jupyterlab`.

### With Google Colab

Google Colab is a free hosted Jupyter Notebook: https://colab.research.google.com/notebooks/welcome.ipynb

It allows you to open a Notebook from GitHub. Paste this Notebook's url into Colab to open it.

In Google Colab, you can update Python packages with pip, for example `pip install pandas --upgrade`.

# Getting started
 
**IMPORTANT**: this quiz uses the Emma corpus from the nltk package. Here is how you can get a copy of it:
 
On the command line:

```
python -c import nltk; nltk.download('gutenberg')
```

Or in a Python session:

```
import nltk
nltk.download('gutenburg')
```

If nltk is not installed, you can install it with

```
pip install nltk
```

-or-

```
conda install nltk
```

If none of this works, you can get the corpus from the "janeaustenr" R package, or you can download it directly: http://www.gutenberg.org/ebooks/158

In [2]:
# You will need the nltk library
import nltk

# You will also need to download the nltk modules (see above). This is
# especially true in a Colab notebook, which forgets all downloads after
# shutdown.
# nltk.download()

In [3]:
# Exercise 0: test your Emma corpus
#
# 
# The full novel
emma_corpus = nltk.corpus.gutenberg.raw('austen-emma.txt')

# A snippet
emma_snippet = emma_corpus[:500]

In [4]:
# Pretty print
print(emma_snippet)

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t


In [5]:
# See what the text really is. This is what you'll be using the regular
# expressions on.
#
# The \n characters you're seeing are newlines. As you can see, they
# separate words, so you'll need to deal with them when parsing.
print(repr(emma_snippet))

"[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.\n\nShe was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister's marriage,\nbeen mistress of his house from a very early period.  Her mother\nhad died t"


# Exercise 1: basic tokenization of the text

In natural language processing, words are known as tokens. The process of splitting words by spaces and punctuation is known as tokenization or parsing. For example "the dog" is tokenized as ["the", "dog"], while "him; however" becomes ["him", "however"] or ["him", ";", "however"]. Sometimes tokenizers will even split "doesn't" into ["does", "n't"].
 
In order to generate text with Markov chains, you'll only need a tokenizer that splits words apart so that they can be operated upon. This can be done with regular expressions.

The Python re module has the re.split() function that lets you use a regular expression to split text into lists.

## Hints

* Import the re module with `import re`
* **Helpful regex concepts**
  * Groups: you can specify various characters as being part of a regex group, for example [abc].
  * Classes : regex comes with pre-built character clases, like \d for all numeric digits.
  * Modifiers: you can modify and quantify your regex elements so that you find the right matches.
* If you want to learn more about tokenizers, have a look at the nltk package's tokenizers: http://www.nltk.org/book/ch03.html#tokenization_index_term or https://www.nltk.org/api/nltk.tokenize.html or https://stackoverflow.com/questions/21361073/tokenize-words-in-a-list-of-sentences-python

**Goal**: parse the `text1` in order to reproduce `split1`.

In [6]:
text1 = "Emma Woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition"
split1 = ['Emma', 'Woodhouse', 'handsome', 'clever', 'and', 'rich', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition']

# Exercise 2: finding the next word(s) of a sequence

When you search a text with CTRL+F, you can find exact matches. With regular expressions, you can find customized patterns. The classic example here would be wanting to find phone numbers in a file. You can do this with the regular expression `\d{3}-\d{4}`. This pattern will match 3 digits, followed by a `-` dash, and then followed with 4 digits.

In the following text fragments, use regular expressions to find the next word in the sequence. In the fifth fragment, use a regular expression to match an unknown word within the fragment.

**Goal**: use regex to match the *next* word

In [7]:
fragment1 = "Emma was now in a"
# a) humour
# b) house
# c) predicament

fragment2 = "with every probability of greater"
# a) injury
# b) happiness
# c) tea

fragment3 = "father does not"
# a) think
# b) see
# c) bathe

fragment4 = "the house of a very old"
# a) man
# b) style
# c) friend

# Trick question: find the missing word within the fragment
fragment5 = "nearly _____ from the vessel"
# a) fallen
# b) dashed
# c) dropped

# Exercise 3: find the most common match to a pattern

By outputting your regex matches to a list, you can perform further operations on them. With Markov chain text generation, you'll want to randomly sample from these possibilities to create a semi-convincing fascimile.

With the fragments below, use regular expressions to help find the *most* frequent continuation of the possibilities provided.

## Hints

* The re.findall() and re.finditer() functions let you output results to a list or an iterable, respectively.
* The Python standard library has ways of counting things. It's also possible to count things using dictionaries.

**Goal**: find all matches and then bring them in for counting

In [8]:
fragment6 = "Emma had"
# What is the most common next word?
# a) never
# b) not
# c) foressen

fragment7 = "a delightful"
# What is the most common next word?
# a) companion
# b) visit
# c) evening

fragment8 = "they had"
# What is the most common next word?
# a) all
# b) been
# c) to

# Exercise 4: build a simple Markov chain text generator

Using what you've learned, construct a loop that does the following

1. extracts the last word of your generated sequence
2. finds all possibilities that may follow this word
3. randomly sample one of these and append it to your text

Repeat the above loop for 250 words.

## Hints

* Have a look at non-capturing positive/negative lookahead/lookbehind. These let you design conditional regex patterns. Because they're non-capturing, they let you peek forwards or backwards without matching.
* The standard Python module random can sample lists for you, and so can the numpy library's random module.

**Goal**: using the Markov chain algorithm described above, generate text from `start`

In [9]:
start = "He was never"

# Bonus 1: an improved Markov chain text generator

Add the following to your text generation script:

1. When choosing the next words in your text, choose them two (or more) at a time instead of one at a time.
   * For example, when completing "dog _____", the script could choose between "was tired", "jumped over", and "barked loudly".
2. When finding matches in the text, match more than one word at a time.
   * For example, look at the exercises above. We tried to complete fragments like "Emma had" and "a delightful".

## Hints

* When trying to match longer sequences in the text, you may not find any match. This will happen when you've generated a weird string of words that don't occur naturally: you won't be able to find continuations.
   * One way of dealing with this is a try-except statement. If the script tries and fails to do something (causes an error), it will jump to the except statement and continue running.
   * Another way of dealing with this situation is a loop with a break statement. Have the script try different lengths of sequences to match. When it succeeds, use the break statement to terminate the loop and move on. This is a better solution than try-except since it doesn't rely on triggering and suppressing an error.
   * You can find more corpora here: https://www.nltk.org/book/ch02.html With more text you might be able to generate more coherent text; however, your script will likely slow down a bit. Using regex for text generation is the most efficient way.

**Goal**: tweak your text generator to (hopefully) generate better text

In [10]:
start = "She was never"